In [1]:
import pandas as pd
import datetime

## CDI

In [29]:
df = pd.read_excel("datalake_files/DCF - RP'22 8&04 - Valores.xlsx",sheet_name = 'Budget',header = 5)
df = df.loc[~df['Unnamed: 1'].isnull()].drop('Unnamed: 0',axis = 1).set_index('Unnamed: 1').T
df.columns.name = None
df = df[list(map(lambda x: type(x) == datetime.datetime,df.index.values))]
serie = df['Cash Accumulated Available'] * 100

## Câmbio

In [30]:
df = pd.read_excel('datalake_files/09 Debt vs Setembro 2022 - RP - Budget 2023.xlsx',sheet_name = 'Daily Calculation prop2019',header = 2)
df = df[['Date','USD','Repayment']].dropna()
df = df.set_index('Date')
df = df[datetime.datetime.today():]
df['Period'] = df.index.to_period('m')
df['Reais_Acumulado'] = (df['USD'] * df['Repayment']).cumsum()
df

1ª Emissão CDI 
1ª Emissão IPCA
2ª Emissão CDI
2ª Emissão IPCA
#REF
3ª Emissão CDI
#REF
#REF
#REF
#REF
#REF
#REF
#REF
#REF
#REF
MUFG 2nd Contract
#REF
#REF
#REF
#REF
#REF


,USD,Repayment,Period,Reais_Acumulado
Date,,,,
2023-05-20,5.073958,7.708333e+07,2023-05,3.911176e+08
2023-11-20,4.947550,7.708333e+07,2023-11,7.724912e+08
2024-05-20,4.907509,7.708333e+07,2024-05,1.150778e+09
2024-11-20,4.916520,7.708333e+07,2024-11,1.529760e+09
2025-05-20,4.911695,7.708333e+07,2025-05,1.908370e+09
2025-11-20,4.904104,7.708333e+07,2025-11,2.286395e+09
2026-05-20,4.723724,0.000000e+00,2026-05,2.286395e+09
